In [12]:
import elevenlabs
import os
import base64
import pygame
import time

pygame 2.6.1 (SDL 2.28.4, Python 3.13.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


c:\Users\rawat\anaconda3\envs\shorts\Lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [3]:
generated_text=input("Enter a text")

In [5]:
from elevenlabs import ElevenLabs

client=ElevenLabs(api_key=os.getenv('ELEVENLABS_API_KEY'))


response=client.text_to_speech.convert_with_timestamps(
    voice_id="NFG5qt843uXKj4pFvR7C",
    text=generated_text
)

In [7]:
audio=base64.b64decode(response.audio_base_64)

with open('test.mp3','wb') as file:
    file.write(audio)

In [8]:
alignment = response.alignment
chars = alignment.characters
starts = alignment.character_start_times_seconds
ends = alignment.character_end_times_seconds

In [9]:
def func(response):
    alignment = response.alignment
    chars = alignment.characters
    starts = alignment.character_start_times_seconds
    ends = alignment.character_end_times_seconds
    words = []
    current_word = ""
    current_start = None

    for c, s, e in zip(chars, starts, ends):
        if c != " ":
            if current_word == "" or (current_word=="." or current_word==","):  # starting a new word
                current_start = s
            current_word += c
            current_end = e
        else:
            if current_word:  # end of a word
                words.append((current_word, current_start, current_end))
                current_word = ""

    # Add the last word if there is one
    if current_word:
        words.append((current_word, current_start, current_end))

    # Print result
    for word, s, e in words:
        print(f"'{word}' spoken from {s:.2f}s to {e:.2f}s")


    return words


In [10]:
timestamp=func(response)

'I' spoken from 0.00s to 0.10s
'am' spoken from 0.17s to 0.29s
'Walter' spoken from 0.35s to 0.66s
'hardwill' spoken from 0.72s to 1.10s
'white' spoken from 1.16s to 1.53s
'i' spoken from 1.59s to 1.70s
'live' spoken from 1.80s to 2.00s
'in' spoken from 2.03s to 2.10s
'panama' spoken from 2.16s to 2.72s
'new' spoken from 2.82s to 3.00s
'mexico' spoken from 3.05s to 3.85s


In [13]:
pygame.mixer.init()
pygame.mixer.music.load("test.mp3")

In [24]:
import ffmpeg

def add_timed_text(stream, text, start, end, fontsize_ratio=0.12, fontcolor="white", outline=6):
    """
    Adds large, bold, centered YouTube Shorts-style text.
    """
    return stream.drawtext(
        text=text,
        fontfile=r"C:\Windows\Fonts\impact.ttf",   # Bold meme/viral look
        fontsize=f"h*{fontsize_ratio}",            # Scale with video height
        fontcolor=fontcolor,
        x="(w-text_w)/2",                          # Center horizontally
        y="(h-text_h)/2",                          # Center vertically
        borderw=outline,                           # Thick outline
        bordercolor="black",
        box=0,                                     # No box, outline only
        enable=f"between(t,{start},{end})"
    )


In [25]:
def overlay(stream,timestamp):
   '''
   Function: Apply timestamp

   Args:
      stream: Canvas to apply work
      timestamp: Words with start and end time
   '''
   for word,start,end in timestamp:
       stream=add_timed_text(stream,word,start,end)
    
   return stream
    


In [26]:
import ffmpeg

input_file = r"C:\Users\rawat\ytshorts\4511381-uhd_2560_1440_30fps.mp4"
audio_file = r"C:\Users\rawat\ytshorts\test.mp3"
output_file = "asscream.mp4"  #mp4 to apply subtitles to

# Load video
video = ffmpeg.input(input_file).video
audio = ffmpeg.input(audio_file).audio

video=overlay(video,timestamp)

(
    ffmpeg
    .output(video, audio, output_file, vcodec='libx264', acodec='aac', strict='experimental')
    .overwrite_output()
    .run()
)


(None, None)